# Pruebas de Ruido con S.T. sintéticas

Los decibeles expresan una relación de poder, no una cantidad. Dicen cuántas veces más (dB positivo) o menos (dB negativo) pero no cuánto en términos absolutos. Los decibeles son logarítmicos, no lineales. Por ejemplo, 20 dB no es el doble de la relación de potencia de 10 dB.

La ecuación para decibelios es:

$A = 10 * log10 (P2 / P1) (dB)$

donde P1 es la potencia que se mide y P1 es la referencia con la que se compara P2.
Para convertir de la medida de decibelios a la relación de potencia:

$P2 / P1 = 10^{(A / 10)}$

In [1]:
import os, sys
sys.path.insert(0, '/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers')
import outliersDetection as od
import requests, pandas as pd, numpy as np
import time, json
import bokeh
from scipy import signal, fft
from statsmodels.tsa.stattools import adfuller, acf, pacf
import statsmodels.graphics as smg
%matplotlib inline
from bokeh.plotting import gridplot, figure, output_file, output_notebook, show
from datetime import datetime
from __future__ import print_function
from PyAstronomy import pyasl
from pylab import *
# Biblioteca para calcular los exponentes de Lyapunov
import nolds 

None


## Valores faltantes

In [132]:
def quitarValores(ts,porcentaje):
    cambios = []
    tam = len(ts)
    elementos = round((porcentaje/100)*tam)
    print("elementos",elementos)
    indices = []
    for i in range(elementos):
        k = 0
        while k == 0:
            r=np.random.randint(tam)
            if r not in indices: 
                indices.append(r)
                k = 1
    print("FALTANTES",indices)
    for i in indices:
        ts[i] = np.nan
    return ts

def valoresFaltantes(nombre,SNR_dB):
    filename = ""
    # Revisando si la st está contaminada con ruido o es la original
    if SNR_dB != -1:
        filename = "st con ruido/"+nombre+str(SNR_dB)+"/ts 0"+".csv"
    else:
        filename = "st originales/"+nombre+".dat"
    tam = len(data)
    num_nans = round(round(data.isnull().sum())/float(tam)*100) 
    print("Porcentaje de valores faltantes",num_nans,"tamano de la serie de tiempo",tam)
    return tam, num_nans

## Outliers

In [137]:
def agregarOutliers(ts,porcentaje):
    cambios = []
    tam = len(ts)
    elementos = round((porcentaje/100)*tam)
    print("elementos",elementos)
    indices = []
    for i in range(elementos):
        k = 0
        while k == 0:
            r=np.random.randint(tam)
            if r not in indices and ts[r] != np.nan: 
                indices.append(r)
                k = 1
    print("OUTLIERS INDICES",indices)
    media = np.mean(ts)
    sigma = np.std(ts)*20
    for i in indices:
        alpha = np.random.rand()
        if alpha >= 0.5:
            ts[i] = media*sigma*alpha
        else:
            ts[i] = media*(-(sigma+alpha))
    return ts

In [141]:
# filename = "st con ruido/seno5/ts 0.csv"
filename = "st originales/seno.dat"
ts_aux = pd.read_csv(filename)
# print(ts_aux[0:20])
ts_aux2 = quitarValores(ts_aux['col1'],5)
plotseries(0,ts=ts_aux2)
# valoresFaltantes(ts_aux2)
ts_aux3 = agregarOutliers(ts_aux2,10)
plotseries(0,ts=ts_aux3)

elementos 10
FALTANTES [1, 77, 45, 7, 61, 91, 97, 18, 15, 32]


Loading BokehJS ...

elementos 20
OUTLIERS INDICES [197, 112, 44, 178, 64, 146, 87, 70, 150, 135, 104, 133, 168, 54, 106, 128, 59, 88, 140, 24]


Loading BokehJS ...

In [128]:
# x = A*np.cos(2*np.pi*1000*t) + a*np.sin(2*np.pi*2000*t) + s*np.random.normal(len(t));
def funcionSeno(fs, f, T=2):
    fs_ = fs #100KHz frecuencia de muestreo
    f_ = f #1KHz frecuencia de la senal
    t_ = np.arange(0,2*(1/f_),1/fs_)
    print(len(t_))
    x_ = np.sin(T*np.pi*f_*t_)
    return x_, t_

def agregarSNR(x,SNR_dB,t_):
    #https://www.gaussianwaves.com/gaussianwaves/wp-content/uploads/2015/06/How_to_generate_AWGN_noise.pdf
    #Matlab / Octave tiene una función incorporada llamada -awgn () con la que se puede agregar 
    #un ruido blanco para obtener la señal deseada a ruido (SNR). 
    #El uso principal de esta función es agregar AWGN a una señal limpia (SNR infinita) para obtener una señal 
    #tamano de la serie
    L = len(x)
    #SNR a escala lineal
    SNR = np.power(10,(SNR_dB/10))
    #Calcular la energía
    Esym = np.sum(np.power(abs(x),2))/(L);
    #Encontrar la densidad espectral de ruido
    N0=Esym/SNR;
    #Desviación estándar para el Ruido
    noiseSigma =np.sqrt(N0);
    #Creando el ruido
    n = np.random.normal(0,noiseSigma,L)
    print("SNR medido en dB",validarSNRinyectado(x, n,t_))
#     print("x\n\n",x,"\n\n\n n \n\n\n",n)
    return x + n

#Funcion para calcular el SNR antes de inyectarlo
def validarSNRinyectado(sig, noise,dt):
    Signal = np.sum(np.abs(np.fft.fft(sig)*dt)**2)/len(np.fft.fft(sig))
    Noise = np.sum(np.abs(np.fft.fft(noise)*dt)**2)/len(np.fft.fft(noise))
    print("SNR medido lineal",Signal/Noise)
    return (10 * np.log10(Signal/Noise))


def obtener_SNR(xn,x=[],t=[],filename='SNR.csv',showPlot=0):
    """
    Parameters:
           xn.- Signal + noise
           x.- Original signal
           t.- time
           filename.- donde guardar los SNR obtenidos
           showPlot.- muestra las graficas con 1 y con 0 no
    Return:
           SNR.- Signal to noise ratio 
    """
    # Estimate noise using robust estimate
    beq = pyasl.BSEqSamp()
    # Define order of approximation (use larger values such as 2,3, or 4 for
    # faster varying or less well sampled data sets; also 0 is a valid order)
    N = 2
    # Define 'jump parameter' (use larger values such as 2,3, or 4 if correlation
    # between adjacent data point is suspected)
    j = 1
    nstd, nstdstd = beq.betaSigma(xn['col1'], N, j, returnMAD=True)
    # Obteniendo la varianza apartir de la desviacion estandar
    var_noise = np.power(nstd,2)
    # Ma va a contener la senal
    Ma = []
    # Res contiene los residuos los cuales se consideran como el ruido
    Res = []
    SNRcalculado = 0
    for window in range(2,20):
        print("Ancho de ventana = ",window)
        # Moving Average Medias Moviles
        Ma = xn['col1'].rolling(window=window,center=True).mean()
        # Residuos de la Media movil
        Res = xn['col1'] - Ma
        # Quitando los 
        Res = Res.dropna()
        # Obteniendo la varianza del ruido
        varianza_Noise_sin = np.var(Res)      
#         restd = np.std(Ma)
        # Obteniendo las frecuencias y el espectro de potencia o densidad espectral de potencia
        fsenal, Pxx_den_senal = signal.periodogram(Ma[~np.isnan(Ma)])
        fnoise, Pxx_den_noise = signal.periodogram(Res[~np.isnan(Res)])
        # Quitando nans de la densidad espectral de potencia tanto para la senal como para el ruido
        Pxx_den_senal = Pxx_den_senal[~np.isnan(Pxx_den_senal)]
        Pxx_den_noise = Pxx_den_noise[~np.isnan(Pxx_den_noise)]

        #Si la diferencia entre la varianza de pyastronomy y la varianza del ruido extraido es menor a 1e-3
        #Podemos decir que se ha encontrado el SNR o que no contiene SNR
        if np.abs(var_noise - varianza_Noise_sin) < 1e-3:
            # Si la ventana es de 2 quiere decir que la serie no contiene ruido
            if window == 2:
                print("No contiene ruido")
                filename.write(",,No SNR\n")
                SNRcalculado = "No SNR"
                break
            # Esta x se utilizo para las pruebas
            if len(x) > 0:
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                filename.write(str(SNRcalculado)+'\n')
                if showPlot:
                    od.grafica2D(t,Ma,t,x,
                             title1='Ruido reducido',title2='Original')
                    od.grafica2D(t,Ma,t,xn['col1'],
                             title1='Ruido reducido',title2='Ruido')
            else:
                # Se calcula el SNR = Psignal/Pnoise donde P es la potencia media https://en.wikipedia.org/wiki/Signal-to-noise_ratio
                SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
                print("\nSe redujo el ruido con una ventana de",window)
                print("SNR detectado",SNRcalculado)
                filename.write(str(SNRcalculado)+'\n')
#                 print("SNR mediaS/desvN",1/(np.mean(Ma)/np.std(Res)))
                if showPlot:
                    od.grafica2D(t,Ma,t,xn['col1'],
                             title1='Noise reduced',title2='Noise signal')
            break
        if window == 19:
            SNRcalculado = 10*np.log10(np.mean(Pxx_den_senal)/np.mean(Pxx_den_noise))
            filename.write(str(SNRcalculado)+',,,Max\n')
    return SNRcalculado


#Funcion para graficar el periodograma
# import matplotlib.pyplot as plt
def plot_periodogram(df_x,title,legend,showPlot=0):
    #Obtenemos la frecuencia y densidad espectral de potencia o espectro de potencia.
    f, Pxx_den = signal.periodogram(df_x.T)
    Pxx_den = np.ravel(Pxx_den)
    Pxx_den[0] = np.nan
    #print(Pxx_den[0:10])
    if showPlot:
        p = bokeh.plotting.figure(title=title, y_axis_type="log",
               background_fill_color="#fafafa")
        p.line(x=f, y=np.ravel(Pxx_den), legend=legend, line_color="blue")
        bokeh.plotting.show(p)
    return f, Pxx_den

def variacion(Pxx_den):
    # Esta funcion recibe el espectro de potencia
    # El indice es el del elemento mas grande
    indice = np.argmax(Pxx_den[~np.isnan(Pxx_den)])
    # Si el indice se encuentra delante del 10% de la posicion quiere decir que no se puede calcular el SNR
    if indice > len(Pxx_den)*0.1:
        return False
    else:
        return True

def plotseries(tipo,path="",ts=[]):
    # Funcion para plotear las series
    if tipo == 0:
        od.grafica2D(np.arange(len(ts)),ts,title1='TS')
    else:
        ts = pd.read_csv(path)
        ts = ts.dropna()
        od.grafica2D(np.arange(len(ts)),ts['col1'],title1='TS')


def crearStRuidosa(nombreTs,cuantasSeries=1):
    #nombreTs: Nombre de la serie de tiempo
    #cuantasSeries: Numero de series independientes
    nivelesSNR = np.arange(5,26)
    for i in nivelesSNR:
        ts = pd.read_csv("st originales/"+nombreTs+".dat")
        ts = ts
        ts = ts.dropna()
        SNR_dB = i
        #print(ts[0:10])
        for j in arange(cuantasSeries):
            # Usando la funcion agregarSNR para inyeectar el ruido
            xn_ = agregarSNR(ts['col1'],SNR_dB,np.arange(len(ts)))
            # Ruta donde se va a guardar la ts
            path = "st con ruido/"+nombreTs+str(SNR_dB)
            # Revisando que exista el directorio si no existe lo crea
            if not os.path.exists(path): 
                os.mkdir(path)
            # Crea el archivo .csv donde se guardara la ts
            csv_file2 = open(path+'/ts '+str(j)+'.csv', "w")
            xn2 = pd.DataFrame({'col0': np.arange(len(xn_)),'col1': xn_})
            csv_file2.write(xn2.to_csv(index=False))
            csv_file2.close()
    print("Termino")

## Outliers

In [27]:
def leerTs(nombre,original=0,SNR_dB=-1):
    ts = pd.DataFrame()
    if original == 1:
        filename = "st originales/"+nombre+".dat"
        ts_aux = pd.read_csv(filename)
        if len(ts_aux) > 20000:
            ts_aux = ts_aux[0:20000]
        ts = pd.DataFrame({'col0': np.arange(len(ts_aux)),'col1': ts_aux['col1']})
    else:
        filename = "st con ruido/"+nombre+str(SNR_dB)+"/ts 0"+".csv"
        ts = pd.read_csv(filename)
        if len(ts) > 20000:
            ts = ts[0:20000]        
#     print(ts)
    Inliers,Outliers,Pos = od.obtenerOutliers(ts,k=20,showPlot=0)
    print(len(Outliers[1]))
    return len(Outliers[1])
leerTs("duffing_oscillator",original=1)

0


0

In [20]:
def SNRTs(nombre,prueba=0,SNR_dB=-1,showPlot=0):
    path = "st con ruido/"+nombre+str(SNR_dB)
    numeroDeSeries = 1
    if prueba:
        numeroDeSeries = 10
    SNRObtenido = 0
    if SNR_dB == -1:
        print("entra",os.path.exists)
        if not os.path.exists:
            print("entra")
            os.mkdir(path)
        numeroDeSeries = 1
    filename = path+"/snr "+nombre+str(SNR_dB)+".csv"
    csv_file = open(filename, "w")  
    columnTitleRow = "SNR detectado, SNR no detectado, No contiene SNR, Prueba con un SNR = "+str(SNR_dB)+'\n'
    csv_file.write(columnTitleRow)
    tiempoGlobal = time.time()
    for i in range(0,numeroDeSeries):
        tiempoInicioTarea = time.time()
        print("i",i)
        if SNR_dB != -1:
            ts = pd.read_csv("st con ruido/"+nombre+str(SNR_dB)+"/ts "+str(i)+".csv")
        else:
            #Read the file 
            ts = pd.read_csv("st originales/"+nombre+".dat")
            ts = ts
            ts = ts.dropna()
#         od.grafica2D(np.arange(len(ts)),ts['col1'],title1=nombre+" SNR "+str(SNR_dB))    
        f_ts, Pxx_den_ts = plot_periodogram(ts,nombre,nombre,showPlot=showPlot)
        varia = variacion(Pxx_den_ts)
        if varia:
            #Obtener el SNR
            SNRObtenido = obtener_SNR(ts,t=np.arange(len(ts)),filename=csv_file,showPlot=showPlot)
        else:
            print("Varia rapidamente")
            SNRObtenido = "Nan"
            csv_file.write(",Nan,\n")
        tiempoFinalTarea = time.time()
        print("Tiempo total ",tiempoFinalTarea-tiempoGlobal,"Tiempo tarea ",tiempoFinalTarea-tiempoInicioTarea)
    columnTitleRow = "=PROMEDIO(A2:A11)\n"
    csv_file.write(columnTitleRow)
    csv_file.close()
    return SNRObtenido

## Caos

In [21]:
def calcularCaos(nombre,SNR_dB=-1):
    # Funcion para calcular el Caos usando la biblioteca nolds y el algoritmo de Rosenstein
#     tiempoInicial = time.time()
    filename = ""
    # Revisando si la st está contaminada con ruido o es la original
    if SNR_dB != -1:
        filename = "st con ruido/"+nombre+str(SNR_dB)+"/ts 0"+".csv"
    else:
        filename = "st originales/"+nombre+".dat"
    # Se carga la ts
    ts = pd.read_csv(filename)
    tamano = len(ts)
    # Se limita a que a lo maximo el tamano de la ts sea 20000 
    if tamano > 20000:
        tamano = 20000
    ts = ts[0:tamano]
#     print("TS\n\n",ts['col1'].shape)
    # Se calcula el exponente
    expo = nolds.lyap_r(np.ravel(ts['col1']))
    #expo = max(nolds.lyap_e(ts['col1']))
#     print("Tiempo total ",time.time()-tiempoInicial)
    return expo

In [28]:
import datetime
# Nombre de la serie de tiempo
# nameTs = "simplest_linear_flow"
# Graficar la serie de tiempo
# plotseries("st originales/"+nameTs+".dat")
# Crear n series de tiempo con SNR de 5 hasta 25
# crearStRuidosa(nameTs,10)

SC = ["lorenz"]
# SC = ["henon","lorenz","rossler","chen_system","duffing_oscillator","halvorsen_attractor",
#       "rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]

# for i in range(4,5,1):
#     for j in SC:
#         if i == 4:
#             # Calcular SNR para st originales
#             SNRTs(j,showPlot=0)
#         else:
#             # Calcular SNR para st con ruido
#             SNRTs(j,i,showPlot=0)
# SNRTs(SC[0],15,showPlot=1)

tiempoInicial = time.time()
SNR_pasos = np.arange(4,26)
print(SNR_pasos)
for i in SC:
    caos = 0
    filename = "caracteristicas/"+i+str(datetime.datetime.now())+".csv"
    csv_file = open(filename, "w")
    columnTitleRow = "SNR_inyectado,Outliers,SNR,Caos\n"
    csv_file.write(columnTitleRow)
    for j in SNR_pasos:
        if j == 4:
            outliers = leerTs(i,original=1)
            snr = SNRTs(i,showPlot=0)
            caos = calcularCaos(i)
            csv_file.write("0,"+str(outliers)+","+str(snr)+","+str(caos)+'\n')
        else:
            outliers = leerTs(i,original=0,SNR_dB=j)
            snr = SNRTs(i,SNR_dB=j,showPlot=0)
            caos = calcularCaos(i,j)
            csv_file.write(str(j)+","+str(outliers)+","+str(snr)+","+str(caos)+'\n')
#         print(i,j,caos)
        print("Tiempo parcial ",time.time()-tiempoInicial)
    csv_file.close()

print("Tiempo total ",time.time()-tiempoInicial)

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25]
0
entra <function exists at 0x7f62d3cf7c80>
i 0
Ancho de ventana =  2
Ancho de ventana =  3

Se redujo el ruido con una ventana de 3
SNR detectado 53.58835543974475
Tiempo total  12.848841905593872 Tiempo tarea  12.848838329315186


/home/victort/anaconda2/envs/py366/lib/python3.6/site-packages/nolds/measures.py:240: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 5000
  warnings.warn(msg.format(min_tsep), RuntimeWarning)
/home/victort/anaconda2/envs/py366/lib/python3.6/site-packages/nolds/measures.py:13: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.sum((x - y)**2, axis=1))


Tiempo parcial  30.661990642547607


KeyError: 'col0'

### Caos

In [ ]:
#ACT_attractor,"chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"
SC = ["seno","henon","lorenz","rossler","ACT_attractor","chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
# SC = ["henon"]
SNR_pasos = np.arange(4,5)
print(SNR_pasos)
for i in SC:
#     filename = "caos/"+i+" caos"+str(tamano)+".csv"
#     csv_file = open(filename, "w")
#     columnTitleRow = "Caos"+str(tamano)+"\n"
#     csv_file.write(columnTitleRow)
    caos = 0
    filename = "caos/"+i+" caos.csv"
    csv_file = open(filename, "w")
    for j in SNR_pasos:
        if j == 4:
            caos = calcularCaos(i)
            csv_file.write("0,"+str(caos)+'\n')
        else:
            caos = calcularCaos(i,j)
            csv_file.write(str(j)+","+str(caos)+'\n')
#         print(i,j,caos)
    csv_file.close()

### Periodograma

Cualquier serie de tiempo puede expresarse como una combinación de ondas de coseno (o seno) con diferentes períodos (cuánto tiempo lleva completar un ciclo completo) y amplitudes (valor máximo / mínimo durante el ciclo). Este hecho se puede utilizar para examinar el comportamiento periódico (cíclico) en una serie de tiempo.

Un periodograma se utiliza para identificar los períodos dominantes (o frecuencias) de una serie de tiempo. Esta puede ser una herramienta útil para identificar el comportamiento cíclico dominante en una serie, especialmente cuando los ciclos no están relacionados con la estacionalidad mensual o trimestral más común.


En el área de series de tiempo llamada análisis espectral, vemos una serie de tiempo como una suma de ondas de coseno con amplitudes y frecuencias variables. Uno de los objetivos de un análisis es identificar las frecuencias (o períodos) importantes en la serie observada. Una herramienta de partida para hacer esto es el periodograma. El periodograma grafica una medida de la importancia relativa de los posibles valores de frecuencia que podrían explicar el patrón de oscilación de los datos observados.

Supongamos que hemos observado datos en n puntos temporales distintos y, por conveniencia, suponemos que n es par. Nuestro objetivo es identificar frecuencias importantes en los datos. Para continuar con la investigación, consideramos el conjunto de posibles frecuencias $ω_{j} = j/n$ para $j=1, 2, ..., n/2$. Estas se llaman las frecuencias armónicas.

Representaremos las series de tiempo como

$x_{t} = \sum^{n/2}_{j=1}[β_{1} (j_{n}) cos(2πω_{j}t) + β_{2}(j_{n}) sen (2πω_{j}t)]$

Esta es una suma de las funciones de seno y coseno en las frecuencias armónicas. La forma de la ecuación proviene de la identidad dada anteriormente.

Piense en los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$ como parámetros de regresión. Luego hay un total de n parámetros porque permitimos que j se mueva de $1$ a $n/2$. Esto significa que tenemos n puntos de datos y n parámetros, por lo que el ajuste de este modelo de regresión será exacto.

El primer paso en la creación del periodograma es la estimación de los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. Realmente no es necesario llevar a cabo esta regresión para estimar los parámetros $β_{1}(j/n)$ y $β_{2}(j/n)$. En su lugar, se utiliza la Transformada Rápida de Fourier (FFT).

Una vez estimados los parámetros, definimos


https://newonlinecourses.science.psu.edu/stat510/node/71/

In [67]:
# SC = ["rossler","ACT_attractor","chen_system","duffing_oscillator","halvorsen_attractor","rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
SC = ["seno"]
for k in SC:
    for i in range(5,6):
        for j in range(0,100):
            filename = "st con ruido/"+k+str(i)+"/ts "+str(j)+".csv"
            ts_aux = pd.read_csv(filename)
            # print(ts_aux['col1'])
            ts_aux = ts_aux.rename(index=str, columns={'Unnamed: 0': "col0", "col1": "col1"})
        #     print(ts_aux)
            ts_aux.to_csv(filename,index=False)
        print(i)

5


In [71]:
crearStRuidosa("sennoo",10)

SNR medido lineal 4.978042717902709
SNR medido en dB 6.970586190845118
SNR medido lineal 4.406363629979132
SNR medido en dB 6.440803339352726
SNR medido lineal 3.7945667562060508
SNR medido en dB 5.791621975885686
SNR medido lineal 4.135302465801816
SNR medido en dB 6.165072803757755
SNR medido lineal 4.9744128424558705
SNR medido en dB 6.967418259198706
SNR medido lineal 4.339057548012892
SNR medido en dB 6.373954101243044
SNR medido lineal 4.772632116746964
SNR medido en dB 6.7875795943972195
SNR medido lineal 4.64492402835524
SNR medido en dB 6.669786151357054
SNR medido lineal 4.789913606353571
SNR medido en dB 6.803276802992642
SNR medido lineal 4.403463417784596
SNR medido en dB 6.437943927491161
SNR medido lineal 7.112409997630572
SNR medido en dB 8.52016783757474
SNR medido lineal 7.725771264910762
SNR medido en dB 8.879418459447576
SNR medido lineal 5.892931272057731
SNR medido en dB 7.703313760450667
SNR medido lineal 7.436392792266247
SNR medido en dB 8.713623212569948
SNR m

SNR medido lineal 97.46352773251063
SNR medido en dB 19.888421267973342
SNR medido lineal 129.88423594476293
SNR medido en dB 21.1355644393177
SNR medido lineal 101.45839850148627
SNR medido en dB 20.06288002792576
SNR medido lineal 118.30964666472373
SNR medido en dB 20.730201573282972
SNR medido lineal 125.2195370247562
SNR medido en dB 20.97672093731438
SNR medido lineal 115.85228298683695
SNR medido en dB 20.639045964239344
SNR medido lineal 121.55721199360202
SNR medido en dB 20.84780730650423
SNR medido lineal 108.15173265322008
SNR medido en dB 20.3403348147927
SNR medido lineal 129.48230680849738
SNR medido en dB 21.1221042802801
SNR medido lineal 109.79996614691761
SNR medido en dB 20.406022062141716
SNR medido lineal 148.96288127237088
SNR medido en dB 21.73078063936608
SNR medido lineal 140.8937747673448
SNR medido en dB 21.488918047210156
SNR medido lineal 159.17472265603007
SNR medido en dB 22.018741018274724
SNR medido lineal 136.16614913473566
SNR medido en dB 21.3406915